# Test notebook for evaluation of Measuring Up privacy leakage analysis

In [20]:
from lxml import html
import requests
import urllib2
from bs4 import BeautifulSoup
import csv
from urlparse import urlsplit, urlunsplit
import datetime
import os
import json

In [27]:
# Define Variables

# Get Member Library Names and URLs
dataDLF = 'https://www.diglib.org/members/'
dataARL = 'http://www.arl.org/membership/list-of-arl-members'
dataOCLC = 'http://www.oclc.org/research/partnership/roster.html'

# data file containing the list of libraries from which to retrieve pages
libraryFile = "data_all_library.csv"

# root path for writing out the retrieved files
writePath = "fileStore"
tablePath = "tableStore"

In [ ]:
# set the execution time to use in creating the subdirectory in which the retrieved HTML files will be placed
runTime = datetime.datetime.now().isoformat()

# Limited test of the partial collection of libraries in the CSV file
limitedTest = False

# create the subdirectory where the retrieved files will be saved
currentWriteFilePath = writePath + "/" + runTime
if not os.path.isdir(currentWriteFilePath):
    os.mkdir(currentWriteFilePath)

# create the subdirectory where the generated output tables will be saved
tableWriteFilePath = tablePath + "/" + runTime
if not os.path.isdir(tableWriteFilePath):
    os.mkdir(tableWriteFilePath)

# Connect to the CSV file containing the list of libraries to test
with open(libraryFile, 'rb') as csvfile:
    libraryReader = csv.DictReader(csvfile, delimiter=",", quotechar='"')
    i=0
    outputDict = {}
    for libraryRow in libraryReader:
        i+=1
        print "(" + str(i) + ") " + "Processing: " + libraryRow['school']
        
        # break the provided URL into components so that secure and insecure requests can be generated
        scheme,host,path,query,fragment = urlsplit(libraryRow['url'])
        insecureLink = urlunsplit(('http',host,path,query,fragment))
        secureLink = urlunsplit(('https',host,path,query,fragment))
        print "Insecure URL: " + insecureLink
        print "Secure URL: " + secureLink
        
        # request the insecure page and process its content
        try:
            insecurePageRequest = requests.get(insecureLink, allow_redirects=True)
            pageHeaders = insecurePageRequest.headers
            pageCookies = dict(insecurePageRequest.cookies)
            pageRedirectHist = insecurePageRequest.history
            pageRedirectEndURL = insecurePageRequest.url
            pageHTML = insecurePageRequest.content
            pageSoup = BeautifulSoup(pageHTML,'html.parser')
            imageURLs = []
            scriptURLs = []
            for image in pageSoup.find_all('img'):
                imageURLs.append(image.get('src'))
            for script in pageSoup.find_all('script'):
                scriptURLs.append(script.get('src'))
            if urlsplit(pageRedirectEndURL)[0] == 'https':
                pageSecureRedirect = True
            else:
                pageSecureRedirect = False
            #print pageHeaders
            #print pageCookies
            #print pageRedirectHist
            #print pageRedirectEndURL
            #print pageSecureRedirect
            #print imageURLs
            #print scriptURLs
            # write out the retrieved HTML file
            thisFileWriteFilename = "insecure_" + host + ".html"
            thisFileFullPathWFilename = currentWriteFilePath+'/'+thisFileWriteFilename
            f = open(thisFileFullPathWFilename, 'w')
            f.write(pageHTML)
            f.close()
            # generate the outputDict entry for this web site
            thisDict = {insecureLink: {
                    'pageHeaders': repr(pageHeaders),
                    'pageCookies': pageCookies,
                    'pageRedirectHist': repr(pageRedirectHist),
                    'pageRedirectEndURL': pageRedirectEndURL,
                    'imageURLs': imageURLs,
                    'scriptURLs': scriptURLs
                }}
            #print thisDict
            outputDict.update(thisDict)
        except:
            print "something went wrong - more soon ;)"
        
        # request the secure page and process its content
        try:
            securePageRequest = requests.get(secureLink, allow_redirects=True)
            pageHeaders = securePageRequest.headers
            pageCookies = dict(securePageRequest.cookies)
            pageRedirectHist = securePageRequest.history
            pageRedirectEndURL = securePageRequest.url
            pageHTML = securePageRequest.content
            pageSoup = BeautifulSoup(pageHTML,'html.parser')
            imageURLs = []
            scriptURLs = []
            for image in pageSoup.find_all('img'):
                imageURLs.append(image.get('src'))
            for script in pageSoup.find_all('script'):
                scriptURLs.append(script.get('src'))
            if urlsplit(pageRedirectEndURL)[0] == 'https':
                pageSecureRedirect = True
            else:
                pageSecureRedirect = False
            #print pageHeaders
            #print pageCookies
            #print pageRedirectHist
            #print pageRedirectEndURL
            #print pageSecureRedirect
            #print imageURLs
            #print scriptURLs
            # write out the retrieved HTML file
            thisFileWriteFilename = "secure_" + host + ".html"
            thisFileFullPathWFilename = currentWriteFilePath+'/'+thisFileWriteFilename
            f = open(thisFileFullPathWFilename, 'w')
            f.write(pageHTML)
            f.close()
            # generate the outputDict entry for this web site
            thisDict = {secureLink: {
                    'pageHeaders': repr(pageHeaders),
                    'pageCookies': pageCookies,
                    'pageRedirectHist': repr(pageRedirectHist),
                    'pageRedirectEndURL': pageRedirectEndURL,
                    'imageURLs': imageURLs,
                    'scriptURLs': scriptURLs
                }}
            #print thisDict
            outputDict.update(thisDict)
        except:
            print "something went wrong - more soon ;)"
        
        f = open(tableWriteFilePath + "/result.json", "w")
        f.write(json.dumps(outputDict))
        f.close()
        print
        print "======================================"
        if limitedTest and i>10:
            break

(1) Processing: University of Alabama Libraries
Insecure URL: http://www.lib.ua.edu/
Secure URL: https://www.lib.ua.edu/

(2) Processing: University at Albany, SUNY, Libraries
Insecure URL: http://library.albany.edu/
Secure URL: https://library.albany.edu/

(3) Processing: University of Alberta Libraries
Insecure URL: http://www.library.ualberta.ca/
Secure URL: https://www.library.ualberta.ca/

(4) Processing: University of Arizona Libraries
Insecure URL: http://www.library.arizona.edu/
Secure URL: https://www.library.arizona.edu/

(5) Processing: Arizona State University Libraries
Insecure URL: http://lib.asu.edu/
Secure URL: https://lib.asu.edu/

(6) Processing: Auburn University Libraries
Insecure URL: http://www.lib.auburn.edu/
Secure URL: https://www.lib.auburn.edu/

(7) Processing: Boston College Libraries
Insecure URL: http://www.bc.edu/libraries/
Secure URL: https://www.bc.edu/libraries/

(8) Processing: Boston Public Library
Insecure URL: http://www.bpl.org/
Secure URL: https: